### Boruta
Boruta is an all relevant feature selection method. It is a wrapper built around the random forest classification algorithm. It tries to capture all the important, interesting features you might have in your dataset with respect to an outcome variable.

<ul>
<li>First, it duplicates the dataset, and shuffle the values in each column. These values are called shadow features. * Then, it trains a classifier, such as a Random Forest Classifier, on the dataset. By doing this, you ensure that you can an idea of the importance -via the Mean Decrease Accuracy or Mean Decrease Impurity- for each of the features of your data set. The higher the score, the better or more important.</li>
<li>Then, the algorithm checks for each of your real features if they have higher importance. That is, whether the feature has a higher Z-score than the maximum Z-score of its shadow features than the best of the shadow features. If they do, it records this in a vector. These are called a hits. Next,it will continue with another iteration. After a predefined set of iterations, you will end up with a table of these hits</li>
<li>At every iteration, the algorithm compares the Z-scores of the shuffled copies of the features and the original features to see if the latter performed better than the former.</li>
</ul>

#### Selecting features from unscaled data

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import sklearn 
from sklearn import preprocessing 

In [4]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
data = pd.read_csv(url)
data['date'] = pd.to_datetime(data['date'])
data['year'], data['month'] , data['time'] , data['hour'] ,data['day'] , data['day_of_week'],data['Numerical_Week'] = data['date'].dt.year, data['date'].dt.month , data['date'].dt.time , data['date'].dt.hour , data['date'].dt.day , data['date'].dt.weekday_name,data['date'].dt.weekday
data['weekStatus'] = data['date'].dt.dayofweek
data['WeekStatus'] = np.where(data['weekStatus'] < 5, 'Weekday', 'Weekend')
#data['DateStr'] = data['date'].apply(lambda x: x.strftime('%Y%m%d%H%M'))
d = data.date[0:len(data.date)]
data_final = []
#print(data)
for i in range (len(d)):
    if (i==0):
        a = 61200
        data_final.append(a)
    elif (i > 0 ):
        a = a + 600
        data_final.append(a)
#print((data))
#df['data_converted'] = data
data['NSM'] = pd.DataFrame({'NSM' : data_final})

data.head(5)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,year,month,time,hour,day,day_of_week,Numerical_Week,weekStatus,WeekStatus,NSM
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,2016,1,17:00:00,17,11,Monday,0,0,Weekday,61200
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,2016,1,17:10:00,17,11,Monday,0,0,Weekday,61800
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,2016,1,17:20:00,17,11,Monday,0,0,Weekday,62400
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,2016,1,17:30:00,17,11,Monday,0,0,Weekday,63000
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,2016,1,17:40:00,17,11,Monday,0,0,Weekday,63600


In [4]:
data1 = data.set_index('date')
data1 = data1.drop('Appliances', 1)
data1 = data1.drop('day_of_week',1)
data1 = data1.drop('WeekStatus',1)
data1 = data1.drop('time',1)
data1.head(5)

# load X and y
X = data1.values
y = data.Appliances.values

In [5]:
from boruta import BorutaPy
 

rf = RandomForestRegressor(n_jobs=-1, max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features - 5 features should be selected
feat_selector.fit(X, y)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X)
print(X_filtered)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	5
Tentative: 	15
Rejected: 	14


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 100
Confirmed: 	5
Tentative: 	15
Rejected: 	14


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 100
Confirmed: 	5
Tentative: 	15
Rejected: 	14


...

C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	97 / 100
Confirmed: 	13
Tentative: 	6
Rejected: 	15


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	98 / 100
Confirmed: 	13
Tentative: 	6
Rejected: 	15
Iteration: 	99 / 100
Confirmed: 	13
Tentative: 	6
Rejected: 	15


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	13
Tentative: 	5
Rejected: 	15
[[  3.00000000e+01   1.92000000e+01   4.47900000e+01 ...,   5.30000000e+00
    1.70000000e+01   6.12000000e+04]
 [  3.00000000e+01   1.92000000e+01   4.47225000e+01 ...,   5.20000000e+00
    1.70000000e+01   6.18000000e+04]
 [  3.00000000e+01   1.92000000e+01   4.46266667e+01 ...,   5.10000000e+00
    1.70000000e+01   6.24000000e+04]
 ..., 
 [  1.00000000e+01   2.56285714e+01   4.27685714e+01 ...,   1.32666667e+01
    1.70000000e+01   1.19004000e+07]
 [  1.00000000e+01   2.54140000e+01   4.30360000e+01 ...,   1.32333333e+01
    1.70000000e+01   1.19010000e+07]
 [  1.00000000e+01   2.52642857e+01   4.29714286e+01 ...,   1.32000000e+01
    1.80000000e+01   1.19016000e+07]]


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


In [6]:
print ('\n Initial features: ', data1.columns.tolist() )
# number of selected features
print ('\n Number of selected features:')
print (feat_selector.n_features_)


 Initial features:  ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2', 'year', 'month', 'hour', 'day', 'Numerical_Week', 'weekStatus', 'DateStr']

 Number of selected features:
16


In [8]:
feature_df = pd.DataFrame(data1.columns.tolist(), columns=['features'])
feature_df
feature_df['rank']=feat_selector.ranking_
feature_df = feature_df.sort_values('rank', ascending=True).reset_index(drop=True)
print ('\n Top %d features:' % feat_selector.n_features_)
print (feature_df.head(feat_selector.n_features_))

#print ('\n Feature ranking:')
#print (feat_selector.ranking_)


 Top 16 features:
       features  rank
0        lights     1
1          hour     1
2     Tdewpoint     1
3        RH_out     1
4   Press_mm_hg     1
5            T9     1
6            T8     1
7          RH_7     1
8          RH_5     1
9       DateStr     1
10         RH_4     1
11           T5     1
12         RH_2     1
13           T3     1
14         RH_3     1
15           T2     1


In [9]:
# check selected features
print ('\n Selected features:')
print (feat_selector.support_)


 Selected features:
[ True False False  True  True  True  True False  True  True  True False
 False False  True  True False  True False False  True  True False False
  True False False False False  True False False False  True]


In [10]:
#check weak features
print ('\n Support for weak features:')
print (feat_selector.support_weak_)


 Support for weak features:
[False False False False False False False False False False False False
  True False False False False False  True False False False False False
 False False False False False False False False False False]


### Selecting features by scaling complete data

In [5]:
a1 = data.Appliances
a2 = a1.values
a3 = a2.reshape((len(a2), 1))
min_max_scaler = preprocessing.MinMaxScaler()
appliances_scaler = min_max_scaler.fit_transform(a3)
data['appliances_scaler']  = appliances_scaler
data.head(5)

data1 = data.drop('date', 1)
data1 = data1.drop('Appliances', 1)
data1 = data1.drop('day_of_week',1)
data1 = data1.drop('WeekStatus',1)
data1 = data1.drop('appliances_scaler',1)
data1 = data1.drop('time',1)
data1.head(5)


#scaling data using MinMAX scaler
X_scale = data1.values
X= min_max_scaler.fit_transform(X_scale)
y = data.appliances_scaler.values

C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
from boruta import BorutaPy

rf = RandomForestRegressor(n_jobs=-1, max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features - 5 features should be selected
feat_selector.fit(X, y)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X)
print(X_filtered)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	34
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	5
Tentative: 	16
Rejected: 	13


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 100
Confirmed: 	5
Tentative: 	16
Rejected: 	13


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 100
Confirmed: 	5
Tentative: 	16
Rejected: 	13


...

Iteration: 	97 / 100
Confirmed: 	13
Tentative: 	6
Rejected: 	15


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	98 / 100
Confirmed: 	13
Tentative: 	6
Rejected: 	15


C:\Users\Dhanisha\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	99 / 100
Confirmed: 	13
Tentative: 	6
Rejected: 	15


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	13
Tentative: 	5
Rejected: 	15
[[  4.28571429e-01   2.25345287e-01   6.84037867e-01 ...,   5.38461538e-01
    7.39130435e-01   0.00000000e+00]
 [  4.28571429e-01   2.25345287e-01   6.82139844e-01 ...,   5.33936652e-01
    7.39130435e-01   2.40378722e-06]
 [  4.28571429e-01   2.25345287e-01   6.79445121e-01 ...,   5.29411765e-01
    7.39130435e-01   4.80758172e-06]
 ..., 
 [  1.42857143e-01   6.92651182e-01   6.27197622e-01 ...,   8.98944193e-01
    7.39130435e-01   9.99985577e-01]
 [  1.42857143e-01   6.77053550e-01   6.34717406e-01 ...,   8.97435897e-01
    7.39130435e-01   9.99987981e-01]
 [  1.42857143e-01   6.66170515e-01   6.32901731e-01 ...,   8.95927602e-01
    7.82608696e-01   1.00000000e+00]]


In [15]:
print ('\n Initial features: ', data1.columns.tolist() )
# number of selected features
print ('\n Number of selected features:')
print (feat_selector.n_features_)


 Initial features:  ['lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'rv1', 'rv2', 'year', 'month', 'hour', 'day', 'Numerical_Week', 'weekStatus', 'DateStr']

 Number of selected features:
13


In [16]:
feature_df = pd.DataFrame(data1.columns.tolist(), columns=['features'])
feature_df
feature_df['rank']=feat_selector.ranking_
feature_df = feature_df.sort_values('rank', ascending=True).reset_index(drop=True)
print ('\n Top %d features:' % feat_selector.n_features_)
print (feature_df.head(feat_selector.n_features_))



 Top 13 features:
     features  rank
0      lights     1
1        hour     1
2   Tdewpoint     1
3      RH_out     1
4          T8     1
5        RH_7     1
6        RH_5     1
7        RH_4     1
8     DateStr     1
9          T2     1
10       RH_3     1
11         T3     1
12       RH_2     1
